### Import Packages


In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
from tensorflow import summary
import tensorflow as tf

In [ ]:
import os
import csv
import numpy as np
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

# Module for Google Drive
from google.colab import drive

# Module for Importing Images
from PIL import Image 

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

print(torch.__version__)

1.7.0+cu101


### Import your drive's contents!

In [ ]:
drive.mount('/content/drive')

KeyboardInterrupt: ignored

### Let's define some path, and our PokeMon dataset
- Put the "pokemon" folder to somewhere of your Google Drive, and define the path to "data_path"
- To 'model_dir', put the drive's directory path that you want to save your model

In [ ]:
data_path = './drive/MyDrive/Dataset/pokemon' #./drive/MyDrive/Path/To/PokeMon/Which/Contains/Train/And/Validate
model_dir = './drive/MyDrive/Codes/models'    #./drive/MyDrive/Path/To/Save/Your/Model

In [ ]:
class PokemonDataset(Dataset):
    def __init__(self, data_path, is_training):
        self.data_path = data_path
        self.train_path = os.path.join(data_path, 'train')
        self.val_path = os.path.join(data_path, 'validate')
        self.is_training = is_training
        if self.is_training:
            self.target_path = self.train_path
        else:
            self.target_path = self.val_path

        self.classes = sorted(os.listdir(self.target_path))
        self.img_path_label = list()

        for c in self.classes:
            img_list = os.listdir(os.path.join(self.target_path, c))
            for fp in img_list:
                full_fp = os.path.join(self.target_path, c, fp)
                self.img_path_label.append((full_fp, c, self.classes.index(c)))
            
        # Add some tranforms for data augmentation.
        self.tensor_transform = torchvision.transforms.ToTensor()
        self.normalize_transform = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                    std=[0.229, 0.224, 0.225])
        self.random_crop = torchvision.transforms.RandomCrop(size = 170)
        self.random_flip = torchvision.transforms.RandomHorizontalFlip(p=0.5)
        self.resize = torchvision.transforms.Resize(size=224)
        self.train_transform = torchvision.transforms.Compose([self.tensor_transform,
                                                            #    self.random_crop,
                                                         self.random_flip,
                                                         self.resize,
                                                         self.normalize_transform])
        self.validate_transform = torchvision.transforms.Compose([self.tensor_transform,
                                                                  self.normalize_transform])

    def __len__(self):
        return len(self.img_path_label)

    def __getitem__(self, idx):
        (fp, class_name, class_label) = self.img_path_label[idx]
        img = Image.open(fp)
        original_img = self.tensor_transform(img)

        if self.is_training:
            input = self.train_transform(img)
        else:
            input = self.validate_transform(img)
            
        sample = dict()
        sample['input'] = input
        sample['original_img'] = original_img
        sample['target'] = class_label
        sample['class_name'] = class_name

        return sample

### Set DataSet and DataLoader

In [ ]:
batch_size = 64

train_dataset = PokemonDataset(data_path, True)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
val_dataset = PokemonDataset(data_path, False)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False, pin_memory=True)

num_classes = 18

### Take a sample and try to look at the one

In [ ]:
sample = next(iter(train_dataloader))

In [ ]:
fig, ax = plt.subplots(1, 7, figsize=(20, 10))
for i in range(7):
    ax[i].imshow(sample['input'][i].permute(1, 2, 0))
    ax[i].set_title(sample['class_name'][i])

### Choose your device - use GPU or not?

In [ ]:
# device = 'cpu'
device = 'cuda'
print('Current Device : {}'.format(device))

### Define the model

In [ ]:
class Model(nn.Module):
    def __init__(self, feat_dim = 2048, dim_output=18):
        super(Model, self).__init__()

        self.dim_output = dim_output
        self.feat_dim = feat_dim

        self.conv1 = nn.Conv2d(feat_dim, feat_dim, kernel_size=1)
        self.fc1 = nn.Linear(feat_dim, feat_dim//4) # 2048 -> 512
        self.fc2 =  nn.Linear(feat_dim//4, feat_dim//8)
        self.fc3 =  nn.Linear(feat_dim//8, dim_output)
        self.relu = nn.LeakyReLU(0.1, inplace=True)
        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.5)
        
        
        self.backbone = torch.hub.load('pytorch/vision:v0.6.0', 'resnext50_32x4d', pretrained=True)
        # # Fix Initial Layers
        for p in list(self.backbone.children())[:-5]:
            p.requires_grad = False
        # # get the structure until the last FC layer
        modules = list(self.backbone.children())[:-1]
        
        self.backbone = nn.Sequential(*modules)
    
    def forward(self, img):
        batch_size = img.shape[0]
        x = self.backbone(img)
        x = self.relu(self.conv1(x))
        x = self.dropout1(x)
        x = self.relu(self.fc1(x.view(batch_size, -1)))
        x = self.dropout2(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

### Create a model and its optimizer


In [ ]:
model = Model()
model = model.to(device)

optimizer = optim.AdamW(model.parameters(), lr=1e-4)

In [ ]:
model(sample['input'].to(device)).shape

### Define functions for train/validation

In [ ]:
def train(model, optimizer, sample):
    model.train()

    criterion = nn.CrossEntropyLoss()

    optimizer.zero_grad()

    input = sample['input'].float().to(device)
    target = sample['target'].long().to(device) 
    
    pred = model(input)
    pred_loss = criterion(pred, target)
    
    top3_val, top3_idx = torch.topk(pred, 3)

    num_correct = torch.sum(top3_idx == target.view(-1, 1))
    
    pred_loss.backward()
       
    optimizer.step()

    return pred_loss.item(), num_correct.item()

In [ ]:
def validate(model, sample):
    model.eval()

    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        input = sample['input'].float().to(device)
        target = sample['target'].long().to(device) 

        pred = model(input)
        pred_loss = criterion(pred, target)

        top3_val, top3_idx = torch.topk(pred, 3)

        num_correct = torch.sum(top3_idx == target.view(-1, 1))

    return pred_loss.item(), num_correct.item()

### Prepare the Tensorboard

In [ ]:
train_log_dir = './runs/train'
train_summary_writer = summary.create_file_writer(train_log_dir)
val_log_dir = './runs/validate'
val_summary_writer = summary.create_file_writer(val_log_dir)

In [ ]:
%tensorboard --logdir runs

### Run Training

In [ ]:
max_epoch = 200
save_stride = 10
tmp_path = './checkpoint.pth'
max_accu = -1
for epoch in tqdm(range(max_epoch)):        
    ###Train Phase
    
    # Initialize Loss and Accuracy
    train_loss = 0.0
    train_accu = 0.0

    # Load the saved MODEL AND OPTIMIZER after evaluation.
    if epoch > 0:
        checkpoint = torch.load(tmp_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        # how about learning rate scheduler?

    # Iterate over the train_dataloader
    with tqdm(total=len(train_dataloader)) as pbar:
        for idx, sample in enumerate(train_dataloader):
            curr_loss, num_correct = train(model, optimizer, sample)
            train_loss += curr_loss / len(train_dataloader)
            train_accu += num_correct / len(train_dataset)
            pbar.update(1)

    # Write the current loss and accuracy to the Tensorboard
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss, step=epoch)                
        tf.summary.scalar('accuracy', train_accu, step=epoch)                

    # save the model and optimizer's information before the evaulation
    checkpoint = {
        'model' : Model(),
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    
    # Save the checkpoint - you can try to save the "best" model with the validation accuracy/loss
    torch.save(checkpoint, tmp_path)
    if (epoch+1) % save_stride == 0:
        torch.save(checkpoint, os.path.join(model_dir, 'pokemon_{}.pth'.format(epoch+1)))
    torch.save(checkpoint, os.path.join(model_dir, 'pokemon_recent.pth'))
    
    ### Validation Phase
    # Initialize Loss and Accuracy
    val_loss = 0.0
    val_accu = 0.0

    # Iterate over the val_dataloader
    with tqdm(total=len(val_dataloader)) as pbar:
        for idx, sample in enumerate(val_dataloader):
            curr_loss, num_correct = validate(model, sample)
            val_loss += curr_loss / len(val_dataloader)
            val_accu += num_correct / len(val_dataloader)
            pbar.update(1)

    # Write the current loss and accuracy to the Tensorboard
    with val_summary_writer.as_default():
        tf.summary.scalar('loss', val_loss, step=epoch)
        tf.summary.scalar('accuracy', val_accu, step=epoch) 

    max_accu = max(val_accu, max_accu)
    if max_accu == val_accu:
        # Save your best model to the checkpoint
        torch.save(checkpoint, os.path.join(model_dir, 'pokemon_best.pth'))

    # These Lines would make you update your Google Drive after the saving.
    drive.flush_and_unmount()
    drive.mount('/content/drive')

    print(train_accu, val_accu)

In [ ]:
drive.mount('/content/drive')

In [ ]:
from google.colab import drive
drive.flush_and_unmount()